In [1]:
%%capture
%pip install torch pandas lightning trl

import torch
from torch import nn
import pytorch_lightning as pl
from datasets import load_dataset, Dataset, DatasetDict
import pandas as pd

import numpy as np
from torch.utils.data import DataLoader, TensorDataset
from transformers import BartForConditionalGeneration, BartTokenizer

In [2]:
SEED = 999
torch.manual_seed(SEED)

if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

#set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Device:", device)


Device: cuda


In [10]:
import pandas as pd
from torch.utils.data import DataLoader, Dataset
import torch
import torch.nn.functional as F

# Custom Dataset
class TextHexDataset(Dataset):
    def __init__(self, dataframe):
        self.dataframe = dataframe

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        text = self.dataframe.iloc[idx]['text_hex']
        text = [int(x, 16) for x in text]

        #pad to reach 256
        padded_text = text + [20] * (256 - len(text))
        tensor_text = torch.tensor(padded_text, dtype=torch.float)

        # Convert hex into tensor
        hex_data = self.dataframe.iloc[idx]['deflate_hex']
        hex_data = [int(x, 16) for x in hex_data]

        #pad to reach 256
        padded_hex_data = hex_data + [20] * (256 - len(hex_data))
        tensor_hex_data = torch.tensor(padded_hex_data, dtype=torch.float)
        return tensor_text, tensor_hex_data

# Load the dataset
df = pd.read_csv('shorthex2hex.csv')

# Create datasets
dataset = TextHexDataset(df)

# Split the dataset
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

# DataLoaders with batch_size = 1
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers = 4)
val_dataloader = DataLoader(val_dataset, batch_size=16, num_workers = 4)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [45]:
class Transformer(pl.LightningModule):
    def __init__(self, input_dim=256, hidden_dim=256, output_dim=256, num_heads=4, num_layers=2, dropout_rate=0.1, learning_rate=0.1):
        super().__init__()
        self.attention = nn.MultiheadAttention(input_dim, num_heads, dropout=dropout_rate)
        self.fc = nn.Linear(input_dim, output_dim)
        self.learning_rate = learning_rate
        self.loss = nn.MSELoss()

    def forward(self, x):
        x = x.unsqueeze(0)  # Add a sequence dimension
        x, _ = self.attention(x, x, x)
        x = x.squeeze(0)  # Remove the sequence dimension
        x = self.fc(x)
        return x

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=self.learning_rate)
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
        return [optimizer], [scheduler]

    def training_step(self, batch, batch_idx):
        x, y = batch
        x.to(device)
        y.to(device)
        y_hat = self(x)
        loss = self.loss(y_hat, y)

        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        x.to(device)
        y.to(device)
        y_hat = self(x)
        loss = self.loss(y_hat, y)
        print('val_loss', loss)

# Create an instance of the Transformer model
model = Transformer().to(device)

# Define the trainer with 50 epochs and showing eval results every 10 epochs
trainer = pl.Trainer(max_epochs=50, enable_checkpointing=False, logger=False)

# Train the model ⚡
trainer.fit(model, train_dataloader, val_dataloader)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name      | Type               | Params
-------------------------------------------------
0 | attention | MultiheadAttention | 263 K 
1 | fc        | Linear             | 65.8 K
2 | loss      | MSELoss            | 0     
-------------------------------------------------
328 K     Trainable params
0         Non-trainable params
328 K     Total params
1.316     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

val_loss tensor(353.2734, device='cuda:0')
val_loss tensor(353.0620, device='cuda:0')


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

val_loss tensor(2.8135, device='cuda:0')
val_loss tensor(3.2028, device='cuda:0')
val_loss tensor(2.6498, device='cuda:0')
val_loss tensor(4.5146, device='cuda:0')
val_loss tensor(3.0454, device='cuda:0')
val_loss tensor(3.6043, device='cuda:0')
val_loss tensor(3.3455, device='cuda:0')
val_loss tensor(3.6902, device='cuda:0')
val_loss tensor(3.5293, device='cuda:0')
val_loss tensor(2.8517, device='cuda:0')
val_loss tensor(3.2813, device='cuda:0')
val_loss tensor(3.2321, device='cuda:0')
val_loss tensor(2.6631, device='cuda:0')
val_loss tensor(2.7227, device='cuda:0')
val_loss tensor(2.9270, device='cuda:0')
val_loss tensor(3.2565, device='cuda:0')
val_loss tensor(3.0191, device='cuda:0')
val_loss tensor(3.3347, device='cuda:0')
val_loss tensor(3.4510, device='cuda:0')
val_loss tensor(2.7700, device='cuda:0')
val_loss tensor(3.7183, device='cuda:0')
val_loss tensor(3.1361, device='cuda:0')
val_loss tensor(2.6090, device='cuda:0')
val_loss tensor(3.9296, device='cuda:0')
val_loss tensor(

Validation: |          | 0/? [00:00<?, ?it/s]

val_loss tensor(2.8135, device='cuda:0')
val_loss tensor(3.2028, device='cuda:0')
val_loss tensor(2.6498, device='cuda:0')
val_loss tensor(4.5146, device='cuda:0')
val_loss tensor(3.0454, device='cuda:0')
val_loss tensor(3.6043, device='cuda:0')
val_loss tensor(3.3455, device='cuda:0')
val_loss tensor(3.6902, device='cuda:0')
val_loss tensor(3.5293, device='cuda:0')
val_loss tensor(2.8517, device='cuda:0')
val_loss tensor(3.2813, device='cuda:0')
val_loss tensor(3.2321, device='cuda:0')
val_loss tensor(2.6631, device='cuda:0')
val_loss tensor(2.7227, device='cuda:0')
val_loss tensor(2.9270, device='cuda:0')
val_loss tensor(3.2565, device='cuda:0')
val_loss tensor(3.0191, device='cuda:0')
val_loss tensor(3.3347, device='cuda:0')
val_loss tensor(3.4510, device='cuda:0')
val_loss tensor(2.7700, device='cuda:0')
val_loss tensor(3.7183, device='cuda:0')
val_loss tensor(3.1361, device='cuda:0')
val_loss tensor(2.6090, device='cuda:0')
val_loss tensor(3.9296, device='cuda:0')
val_loss tensor(

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=50` reached.



val_loss tensor(4.2656, device='cuda:0')
val_loss tensor(3.6577, device='cuda:0')
val_loss tensor(3.3565, device='cuda:0')
val_loss tensor(3.6900, device='cuda:0')
val_loss tensor(3.1247, device='cuda:0')
val_loss tensor(3.3413, device='cuda:0')


In [37]:
import nltk
from nltk.metrics.distance import edit_distance

model.to('cpu')

def decimal_to_hexadecimal(decimal):
    hex_digits = "0123456789abcdefghilmnopqrstuvzppppppppppppppppppppp"
    return hex_digits[decimal]

hex_predictions = []
hex_golds = []
scores = []

for text, gold in val_dataloader:

    prediction = model.forward(text).tolist()[0]

    #round every elem in prediction
    prediction = [round(x) for x in prediction]

    #set to 0 every negative elem in prediction
    prediction = [max(0, x) for x in prediction]

    #convert every 0 to 0, 1 to 1, ... , 15 to f
    hex_prediction = [decimal_to_hexadecimal(x) for x in prediction]

    gold = [round(x) for x in gold[0].tolist()]

    hex_gold = [decimal_to_hexadecimal(x) for x in gold]

    #convert to string
    hex_prediction = ''.join(hex_prediction)
    hex_gold = ''.join(hex_gold)

    print(hex_prediction)
    print(hex_gold)

    hex_predictions.append(hex_prediction)
    hex_golds.append(hex_gold)

    scores.append(edit_distance(hex_prediction, hex_gold))

# Convert hex strings to integers
print(f"Average distance is {np.mean(scores)}")

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


...
Average distance is 57.552


Average distance is 57.6128
